In [ ]:
# to evaluate the error
# negative error never happens
%matplotlib inline
import matplotlib.pyplot as plt
import subprocess
import os
import random
import shutil
from collections import Counter
import statistics

DATASETS_DIR = "../tests/input_files/"
# different sets for precision and performance tests
DATASETS = ['test_100_25_15', 'test_100_25_25', 'test_100_25_50', 'test_100_25_75', 'test_100_50_15',
            'test_100_50_25', 'test_100_50_50', 'test_100_50_75']
print(DATASETS)

In [ ]:
# run and collect data
CMD_TEMPL = "../CSP {} {}"
dataset_data = {}


def get_answer(lo, hi, filename):
    """Get the smallest program answer."""
    for k_ in range(lo, hi + 1):
        cmd = CMD_TEMPL.format(filename, k_)
        answer = subprocess.check_output(cmd, shell=True).decode("utf-8").split("\n")[1]
        if answer == "False":
            continue
        return k_
        

datasets_num = len(DATASETS)
for num, dataset in enumerate(DATASETS):
    print("Dataset {} / {} in progress".format(num + 1, datasets_num))
    dataset_metadata = dataset.split("_")
    str_len = int(dataset_metadata[1])
    str_num = int(dataset_metadata[2])
    answer = int(dataset_metadata[3])
    dataset_data[num] = {"answers": [], 
                         "data": {"str_len": str_len,
                                  "str_num": str_num,
                                  "answer": answer}}
    dataset_dir = os.path.join(DATASETS_DIR, dataset)
    contents = os.listdir(dataset_dir)
    for fle in contents:
        f_path = os.path.join(dataset_dir, fle)
        k = get_answer(answer, str_len, f_path)
        dataset_data[num]["answers"].append(k)

In [ ]:
for k, v in dataset_data.items():
    v["errors"] = [a - v["data"]["answer"] for a in v["answers"]]

In [ ]:
# visualize the error
fig = plt.figure(figsize=(20, 10))
rows = 1
cols = 8
to_plot = []
titles = []
ax = fig.add_subplot(rows, cols, num)
for num, (k, v) in enumerate(dataset_data.items(), 1):
    # ax = fig.add_subplot(rows, cols, num)
    ans = v["data"]["answer"]
    perc_error = [e / ans * 100 for e in v["errors"]]
    to_plot.append(perc_error)
    title = "{} {} {}".format(v["data"]["str_len"], v["data"]["str_num"], ans)
    titles.append(title)
    # ax.set_title(title)
ax = fig.add_subplot(1, 1, 1)
bp = ax.boxplot(to_plot)
ax.set_ylabel("Error")
ax.set_xlabel("Dataset")
ax.set_xticklabels(titles, rotation=45, fontsize=8)
plt.show()